In [9]:
import numpy as np
from tqdm import tqdm
from organo_simulator.simulator_LJ_force import FastOverdampedSimulator
import napari

In [10]:
np.random.seed(1)

dt = 0.1
d=2 # dimension of simulation
N_part = 500 # number of particles
nuclei_sizes = 1

# persistence times (in physical units) for the OU process
persistence_times = np.array([10]*(N_part))
viscosities = np.array([1]*(N_part))
Ds = np.array([0]*(N_part)) # diffusion coefficients

Nx = 1000 # length of the box in pixels for the rendering 

In [11]:
simulator = FastOverdampedSimulator(
    L=None, # will be initialized automatically
    Nx=Nx, # length of the box in pixels for the rendering 
    d=d, # dimensions (2 or 3)
    N_part=N_part, # number of particles
    nuclei_sizes=nuclei_sizes, # radius of particles
    viscosity=viscosities, # viscosities of particles
    D=Ds, # diffusion coefficients
    persistence_time=persistence_times, # how long the temporal autocorrelation of random movement goes
    energy_potential=1,
    parallel=True
)

Max overall distance: 3.03


In [4]:
skip = 1 # return coordinates every s
total_steps = 1000 # total number of simulation steps
Nt = int(total_steps/skip)

In [5]:
# Initializing arrays to collect data
data = np.empty((N_part*Nt,d+1))
data_points=[]


for i in tqdm(range(total_steps)):
    simulator.update_dynamics(dt=dt)

    if i%skip==0:
        positions = simulator.dump_coordinates()
        time_pos = np.hstack((i/skip * np.ones((N_part,1)), positions))

        #data = np.vstack((data, time_pos))
        data[int(i/skip)*len(time_pos):(int(i/skip)+1)*len(time_pos),:] = time_pos
        data_points.append(positions)

100%|██████████| 1000/1000 [00:08<00:00, 111.17it/s]


In [6]:
np.max(data[:,1:])

236.40175363396713

In [7]:
import numpy as np
tids = np.tile(np.arange(N_part), Nt)
tracks = np.empty((N_part*Nt, 4))
tracks[:,0] = tids
tracks[:,1:] = data

In [8]:
viewer = napari.Viewer()
viewer.add_tracks(tracks)

/home/jvanaret/anaconda3/envs/napari-env/lib/python3.9/site-packages/napari/_qt/qt_viewer.py:395: UserWarning: Attempting to unbind an action which does not exists (napari:select_all), this may have no effects. This can happen if your settings are out of date, if you upgraded napari, upgraded or deactivated a plugin, or made a typo in in your custom keybinding.
  action_manager.unbind_shortcut(action)


<Tracks layer 'tracks' at 0x7f988080ec40>